# Credit Hungry Predict

In [1]:
'''Importing Required Libraries'''
import copy
from collections import OrderedDict
import pandas as pd
import lightgbm as lgb
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.2f}'.format
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from scipy import stats
from scipy.stats import norm, skew
from sklearn.preprocessing import PowerTransformer, QuantileTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier,SGDRegressor
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import make_pipeline
import joblib
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

In [2]:
'''Importing the data for modelling'''
ch_data = pd.read_csv(r"D:\Dbeaver Out\CH_Predict_Data_2304_202304051738.csv", na_values=['',' ','NH', 'NA', '[]'], low_memory=False)

In [3]:
ch_data.drop(['status','report_source', 'report_id', 'report_date', 'filename_prefix', 'pan_no',
        'passport_no', 'driving_license_no', 'voter_id_no', 'ration_card_no',
        'uid_no', 'national_card_no', 'photo_credit_card_no', 'other_id_no',
        'account_type', 'reported_on', 'frequency', 'income_indicator',
        'ability_to_borrow_cc', 'ability_to_borrow_hl', 'ability_to_borrow_pl',
        'ability_to_borrow_al','created_on', 'modified_on', 'modified_by', 'modified_ts','is_active'],axis=1,inplace=True) 

In [4]:
ch_data.annual_income = ch_data.annual_income.apply(lambda x: (x * -1) if x < 0 else x)
ch_data.existing_foir = ch_data.existing_foir.apply(lambda x: (x * -1) if x < 0 else x)

In [5]:
ch_data.total_accounts.fillna(0, inplace= True) 
ch_data.negative_status_accounts.fillna(0, inplace= True) 
ch_data.positive_status_accounts.fillna(0, inplace= True) 
ch_data.open_accounts.fillna(0, inplace= True) 
ch_data.closed_accounts.fillna(0, inplace= True) 
ch_data.max_dpd.fillna(0, inplace= True) 
ch_data.credit_utilization.fillna(0, inplace= True) 
ch_data.enquiries_made.fillna(0, inplace= True) 
ch_data.seasoning.fillna(0, inplace= True) 
ch_data.portfolio_mix_secured.fillna(0, inplace= True) 
ch_data.portfolio_mix_unsecured.fillna(0, inplace= True) 
ch_data.related_accounts.fillna(0, inplace= True) 
ch_data.annual_income.fillna(30000, inplace= True) 
ch_data.calculated_emi.fillna(0, inplace= True) 
ch_data.minor_issues.fillna(0, inplace= True) 
ch_data.ontime_payment_percentage.fillna(0, inplace= True) 
ch_data.no_of_delayed_payments.fillna(0, inplace= True) 
ch_data.no_of_delayed_payments_on_active_accounts.fillna(0, inplace= True) 
ch_data.no_of_active_accounts.fillna(0, inplace= True) 
ch_data.existing_foir.fillna(0, inplace= True) 
ch_data.credit_utilization_avg.fillna(0, inplace= True) 

In [6]:
ch_data['p_credit_score'] = np.where(ch_data['p_credit_score']>775,775,ch_data['p_credit_score'])
ch_data['total_accounts'] = np.where(ch_data['total_accounts']>47,47,ch_data['total_accounts'])
ch_data['negative_status_accounts'] = np.where(ch_data['negative_status_accounts']>2,2,ch_data['negative_status_accounts'])
ch_data['positive_status_accounts'] = np.where(ch_data['positive_status_accounts']>23,23,ch_data['positive_status_accounts'])
ch_data['open_accounts'] = np.where(ch_data['open_accounts']>14,14,ch_data['open_accounts'])
ch_data['closed_accounts'] = np.where(ch_data['closed_accounts']>17,17,ch_data['closed_accounts'])
ch_data['max_dpd'] = np.where(ch_data['max_dpd']>120,120,ch_data['max_dpd'])
ch_data['credit_utilization'] = np.where(ch_data['credit_utilization']>100,100,ch_data['credit_utilization'])
ch_data['enquiries_made'] = np.where(ch_data['enquiries_made']>13,13,ch_data['enquiries_made'])
ch_data['seasoning'] = np.where(ch_data['seasoning']>8,8,ch_data['seasoning'])
ch_data['portfolio_mix_secured'] = np.where(ch_data['portfolio_mix_secured']>1,1,ch_data['portfolio_mix_secured'])
ch_data['portfolio_mix_unsecured'] = np.where(ch_data['portfolio_mix_unsecured']>1,1,ch_data['portfolio_mix_unsecured'])
ch_data['related_accounts'] = np.where(ch_data['related_accounts']>5,5,ch_data['related_accounts'])
ch_data['annual_income'] = np.where(ch_data['annual_income']>560000,560000,ch_data['annual_income'])
ch_data['calculated_emi'] = np.where(ch_data['calculated_emi']>75000,75000,ch_data['calculated_emi'])
ch_data['minor_issues'] = np.where(ch_data['minor_issues']>10,10,ch_data['minor_issues'])
ch_data['no_of_delayed_payments'] = np.where(ch_data['no_of_delayed_payments']>45,45,ch_data['no_of_delayed_payments'])
ch_data['no_of_delayed_payments_on_active_accounts'] = np.where(ch_data['no_of_delayed_payments_on_active_accounts']>45,45,ch_data['no_of_delayed_payments_on_active_accounts'])
ch_data['no_of_active_accounts'] = np.where(ch_data['no_of_active_accounts']>10,10,ch_data['no_of_active_accounts'])
ch_data['existing_foir'] = np.where(ch_data['existing_foir']>3500,3500,ch_data['existing_foir'])
ch_data['credit_utilization_avg'] = np.where(ch_data['credit_utilization_avg']>100,100,ch_data['credit_utilization_avg'])
ch_data['is_saleable'] = np.where(ch_data['is_saleable']>4,4,ch_data['is_saleable'])

In [7]:
ch_data.columns

Index(['user_id', 'customer_profile_id', 'p_credit_score', 'p_occupation',
       'p_equifax_income', 'p_customer_type', 'total_accounts',
       'negative_status_accounts', 'positive_status_accounts', 'open_accounts',
       'closed_accounts', 'max_dpd', 'credit_utilization', 'enquiries_made',
       'seasoning', 'portfolio_mix_secured', 'portfolio_mix_unsecured',
       'related_accounts', 'annual_income', 'calculated_emi', 'minor_issues',
       'ontime_payment_percentage', 'no_of_delayed_payments',
       'no_of_delayed_payments_on_active_accounts', 'no_of_active_accounts',
       'existing_foir', 'credit_utilization_avg', 'is_saleable'],
      dtype='object')

In [8]:
ch_data.annual_income.median()

30000.0

In [9]:
ch_data.shape

(3711888, 28)

In [10]:
ch_data.isnull().sum()

user_id                                            0
customer_profile_id                                0
p_credit_score                                     0
p_occupation                                 3069744
p_equifax_income                             1817195
p_customer_type                                    0
total_accounts                                     0
negative_status_accounts                           0
positive_status_accounts                           0
open_accounts                                      0
closed_accounts                                    0
max_dpd                                            0
credit_utilization                                 0
enquiries_made                                     0
seasoning                                          0
portfolio_mix_secured                              0
portfolio_mix_unsecured                            0
related_accounts                                   0
annual_income                                 

In [11]:
''' get_occupation '''
def get_occupation(occupation):
    try:
        if occupation == 9:
            return 'Student'
        elif occupation in [1,2]:
            return 'Salaried'
        elif occupation in [3,4,5,6,7,8]:
            return 'SelfEmployed'
        elif occupation in [10,11]:
            return 'Other'
        else:
            return 'Other'
    except:
        return 'Other'
ch_data['p_occupation'] = ch_data['p_occupation'].apply(lambda x: get_occupation(x))
ch_data['p_occupation'].value_counts()

Other           3069744
Salaried         419048
SelfEmployed     223096
Name: p_occupation, dtype: int64

In [12]:
''' get_salary '''
def get_salary(salary):
    try:
        if salary == 1:
            return '<10k'
        elif salary == 2:
            return '10-15k'
        elif salary == 3:
            return '15-20k'
        elif salary == 4:
            return '20-25k'
        elif salary == 5:
            return '25-30k'
        elif salary == 6:
            return '30-35k'
        elif salary == 7:
            return '35-40k'
        elif salary == 8:
            return '40-45k'
        elif salary == 9:
            return '45-50k'
        elif salary == 10:
            return '50-55k'
        elif salary == 11:
            return '55-60k'
        elif salary == 12:
            return '60-65k'
        elif salary == 13:
            return '65-70k'
        elif salary ==14:
            return '70-75k'
        elif salary == 15:
            return '>75k'
        else:
            return 'Other'
    except:
        return 'Other'
ch_data['p_equifax_income'] = ch_data['p_equifax_income'].apply(lambda x: get_salary(x))
ch_data['p_equifax_income'].value_counts()

Other     1817195
>75k       469196
15-20k     300745
10-15k     237562
20-25k     202406
25-30k     136306
45-50k     132396
<10k       125523
30-35k      78462
35-40k      62203
50-55k      53379
40-45k      39022
55-60k      27451
70-75k      10150
65-70k      10133
60-65k       9759
Name: p_equifax_income, dtype: int64

In [13]:
''' get_p_customer_type '''
def get_p_customer_type(type):
    try:
        if type == 1:
            return 'Green'
        elif type == 2:
            return 'Red'
        else:
            return 'Amber'
    except:
        return 'Amber'
ch_data['p_customer_type'] = ch_data['p_customer_type'].apply(lambda x: get_p_customer_type(x))
ch_data['p_customer_type'].value_counts()

Green    1995358
Red      1716530
Name: p_customer_type, dtype: int64

In [14]:
joblib.dump(ch_data, 'ch_clean_data_predict.pkl')

['ch_clean_data_predict.pkl']

In [15]:
exclude=[]

In [16]:
numerical_columns=list(ch_data.select_dtypes(include='number').columns)
for feature in numerical_columns:
    if feature in exclude:
        pass
    else:
        numerical_columns.remove(feature)
print(numerical_columns)

['customer_profile_id', 'total_accounts', 'positive_status_accounts', 'closed_accounts', 'credit_utilization', 'seasoning', 'portfolio_mix_unsecured', 'annual_income', 'minor_issues', 'no_of_delayed_payments', 'no_of_active_accounts', 'credit_utilization_avg']


In [17]:
skewed=[]
def find_skew(data,numerical_columns):
    skew_list={}
    for feature in numerical_columns:
        try:
            try:
                skew_value=(3*(data[feature].mean()-data[feature].median()))/data[feature].std()
                if ((round(skew_value,2)<-3) or (round(skew_value,2)>3)):
                    skew_list[feature]='Skewwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwww'
                    skewed.append(feature)
                else:
                    skew_list[feature]=round(skew_value,2)
            except ZeroDivisionError:
                skew_list[feature]='Skewwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwww'
                skewed.append(feature)
        except:
            pass
    return skew_list
skew_list=find_skew(ch_data,numerical_columns)
skew_list

{'customer_profile_id': -0.53,
 'total_accounts': 1.04,
 'positive_status_accounts': 0.73,
 'closed_accounts': 1.11,
 'credit_utilization': 1.49,
 'seasoning': 0.41,
 'portfolio_mix_unsecured': -2.39,
 'annual_income': 0.95,
 'minor_issues': 0.38,
 'no_of_delayed_payments': 1.78,
 'no_of_active_accounts': 0.84,
 'credit_utilization_avg': 1.51}

In [18]:
for i in numerical_columns:
    if i in exclude:
        pass
    else:
        try:
            ch_data[i] =stats.boxcox(array)
        except:
            pass
find_skew(ch_data,numerical_columns)

{'customer_profile_id': -0.53,
 'total_accounts': 1.04,
 'positive_status_accounts': 0.73,
 'closed_accounts': 1.11,
 'credit_utilization': 1.49,
 'seasoning': 0.41,
 'portfolio_mix_unsecured': -2.39,
 'annual_income': 0.95,
 'minor_issues': 0.38,
 'no_of_delayed_payments': 1.78,
 'no_of_active_accounts': 0.84,
 'credit_utilization_avg': 1.51}

In [19]:
joblib.dump(ch_data, 'ch_transformed_data_predict.pkl')

['ch_transformed_data_predict.pkl']

In [20]:
user_ids=pd.concat([ch_data['user_id'].reset_index(drop = True)], axis = 1)
ch_data.drop(['user_id'],axis=1,inplace=True)

In [21]:
customer_profile_id=pd.concat([ch_data['customer_profile_id'].reset_index(drop = True)], axis = 1)
ch_data.drop(['customer_profile_id'],axis=1,inplace=True)

In [22]:
categorical_features=list(ch_data.select_dtypes(exclude='number').columns)
categorical_features

['p_occupation', 'p_equifax_income', 'p_customer_type']

In [23]:
exemption_features=[]

In [24]:
numeric_features=list(ch_data.select_dtypes(include='number').columns)
numeric_features
flag_features=[]
for feature in exemption_features:
    numeric_features.remove(feature)
for feature in exclude:
    if feature in numeric_features:
        flag_features.append(feature)
        numeric_features.remove(feature)

In [25]:
id_columns=['user_id','customer_profile_id']
exemption_features=[]

In [26]:
dummies = pd.get_dummies(data = ch_data[categorical_features], drop_first=False)
dummies.columns

Index(['p_occupation_Other', 'p_occupation_Salaried',
       'p_occupation_SelfEmployed', 'p_equifax_income_10-15k',
       'p_equifax_income_15-20k', 'p_equifax_income_20-25k',
       'p_equifax_income_25-30k', 'p_equifax_income_30-35k',
       'p_equifax_income_35-40k', 'p_equifax_income_40-45k',
       'p_equifax_income_45-50k', 'p_equifax_income_50-55k',
       'p_equifax_income_55-60k', 'p_equifax_income_60-65k',
       'p_equifax_income_65-70k', 'p_equifax_income_70-75k',
       'p_equifax_income_<10k', 'p_equifax_income_>75k',
       'p_equifax_income_Other', 'p_customer_type_Green',
       'p_customer_type_Red'],
      dtype='object')

In [27]:
'''Scaling numeric features'''
standard_scalar = StandardScaler(with_mean=True,with_std=True)
standard_scalar.fit(ch_data[numeric_features])
numeric_data_scaled = standard_scalar.transform(ch_data[numeric_features])
numeric_data_scaled = pd.DataFrame(numeric_data_scaled,columns=numeric_features)
numeric_data_scaled.head()

,p_credit_score,total_accounts,negative_status_accounts,positive_status_accounts,open_accounts,closed_accounts,max_dpd,credit_utilization,enquiries_made,seasoning,portfolio_mix_secured,portfolio_mix_unsecured,related_accounts,annual_income,calculated_emi,minor_issues,ontime_payment_percentage,no_of_delayed_payments,no_of_delayed_payments_on_active_accounts,no_of_active_accounts,existing_foir,credit_utilization_avg,is_saleable
0,-0.01,0.48,-0.85,1.25,0.73,0.92,-0.26,2.47,-0.69,-0.14,-0.53,0.80,-0.54,-0.15,0.91,-0.13,0.79,-0.66,-0.66,1.05,0.56,2.59,-0.55
1,0.60,-0.87,-0.85,-0.99,-0.77,-1.01,-0.26,-0.50,-0.69,-0.95,-0.53,0.80,0.29,-0.32,0.15,-0.13,0.79,-0.66,-0.66,-0.94,-0.48,-0.50,-0.55
2,0.30,-0.05,-0.85,0.38,0.98,-0.37,-0.26,-0.35,-0.69,-0.54,-0.53,0.80,0.29,3.72,-0.48,1.58,0.79,-0.66,-0.66,0.38,-0.48,-0.35,-0.55
3,0.60,-1.02,-0.85,-1.24,-1.27,-1.01,-0.26,-0.50,-0.69,-0.95,-0.53,-1.60,-0.54,-0.32,-0.76,-0.70,-1.93,-0.66,-0.66,-1.27,-0.48,-0.50,-0.55
4,0.60,0.48,0.34,1.12,0.98,0.75,-0.26,1.20,-0.29,0.68,1.27,-0.40,1.94,-0.16,0.84,1.58,0.24,0.86,0.86,0.38,0.53,1.25,0.45


In [28]:
'''Concatenating all features'''
processed_dataset = pd.concat([user_ids['user_id'].reset_index(drop = True), 
                               customer_profile_id['customer_profile_id'].reset_index(drop = True), 
                               numeric_data_scaled.reset_index(drop = True),
                               ch_data[flag_features].reset_index(drop = True),
                               dummies.reset_index(drop = True),
                              ch_data[exemption_features].reset_index(drop = True)
                              ], axis=1)
processed_dataset.shape

(3711888, 46)

In [29]:
processed_dataset['p_customer_type_Amber']=0

In [30]:
joblib.dump(processed_dataset, 'ch_processed_data_predict.pkl')

['ch_processed_data_predict.pkl']

In [31]:
processed_dataset.corr(method='pearson').style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\formats\style.py:3555: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(gmap) if vmin is None else vmin
C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\formats\style.py:3556: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(gmap) if vmax is None else vmax


,user_id,customer_profile_id,p_credit_score,total_accounts,negative_status_accounts,positive_status_accounts,open_accounts,closed_accounts,max_dpd,credit_utilization,enquiries_made,seasoning,portfolio_mix_secured,portfolio_mix_unsecured,related_accounts,annual_income,calculated_emi,minor_issues,ontime_payment_percentage,no_of_delayed_payments,no_of_delayed_payments_on_active_accounts,no_of_active_accounts,existing_foir,credit_utilization_avg,is_saleable,p_occupation_Other,p_occupation_Salaried,p_occupation_SelfEmployed,p_equifax_income_10-15k,p_equifax_income_15-20k,p_equifax_income_20-25k,p_equifax_income_25-30k,p_equifax_income_30-35k,p_equifax_income_35-40k,p_equifax_income_40-45k,p_equifax_income_45-50k,p_equifax_income_50-55k,p_equifax_income_55-60k,p_equifax_income_60-65k,p_equifax_income_65-70k,p_equifax_income_70-75k,p_equifax_income_<10k,p_equifax_income_>75k,p_equifax_income_Other,p_customer_type_Green,p_customer_type_Red,p_customer_type_Amber
user_id,1.0,0.12,0.17,-0.33,-0.24,-0.34,-0.28,-0.35,-0.033,-0.19,-0.013,-0.074,-0.028,-0.038,-0.21,-0.17,-0.23,-0.21,0.016,-0.23,-0.23,-0.25,-0.16,-0.19,-0.28,0.085,-0.076,-0.034,0.002,-0.023,-0.048,-0.05,-0.044,-0.044,-0.04,-0.023,-0.024,-0.028,-0.026,-0.025,-0.029,-0.036,-0.19,0.25,0.21,-0.21,nan
customer_profile_id,0.12,1.0,0.037,0.024,-0.052,0.025,0.023,0.016,0.0017,0.02,0.067,0.025,-0.016,0.0044,-0.015,-0.021,-0.028,0.1,0.011,-0.051,-0.051,-0.017,-0.0047,0.026,-0.048,-0.00065,0.0041,-0.0045,0.0027,0.01,0.0094,0.0067,0.0055,0.0039,0.003,0.024,-0.0022,0.00043,-0.00078,-0.00016,-0.0032,-0.0072,-0.026,-0.0051,0.06,-0.06,nan
p_credit_score,0.17,0.037,1.0,-0.17,-0.65,-0.09,-0.11,-0.18,-0.25,-0.055,-0.031,0.18,0.013,0.29,-0.13,0.0061,-0.071,-0.13,0.46,-0.55,-0.55,-0.084,-0.091,-0.074,-0.46,0.0079,0.0097,-0.025,-0.033,-0.044,-0.049,-0.041,-0.031,-0.024,-0.021,-0.015,-0.025,-0.017,-0.0078,-0.0076,-0.0087,-0.021,-0.02,0.14,0.64,-0.64,nan
total_accounts,-0.33,0.024,-0.17,1.0,0.22,0.92,0.76,0.9,0.072,0.3,0.32,0.53,0.046,0.21,0.36,0.14,0.41,0.6,0.17,0.23,0.23,0.68,0.38,0.29,0.21,-0.052,0.046,0.022,0.045,0.1,0.13,0.12,0.1,0.089,0.078,0.092,0.047,0.05,0.039,0.039,0.033,0.034,0.16,-0.44,-0.17,0.17,nan
negative_status_accounts,-0.24,-0.052,-0.65,0.22,1.0,0.13,0.18,0.23,0.31,0.045,0.022,-0.21,0.044,-0.18,0.15,0.005,0.055,0.19,-0.43,0.74,0.74,0.14,0.095,0.072,0.71,-0.0089,0.0016,0.012,0.044,0.063,0.067,0.053,0.04,0.031,0.026,0.0057,0.016,0.017,0.01,0.0095,0.015,0.027,0.037,-0.18,-0.91,0.91,nan
positive_status_accounts,-0.34,0.025,-0.09,0.92,0.13,1.0,0.75,0.94,0.027,0.31,0.31,0.57,0.06,0.27,0.35,0.17,0.43,0.53,0.28,0.13,0.13,0.69,0.37,0.29,0.12,-0.053,0.048,0.021,0.047,0.1,0.13,0.12,0.097,0.088,0.076,0.097,0.05,0.05,0.039,0.039,0.033,0.042,0.19,-0.46,-0.08,0.08,nan
open_accounts,-0.28,0.023,-0.11,0.76,0.18,0.75,1.0,0.59,0.095,0.42,0.37,0.75,0.02,0.34,0.34,0.22,0.53,0.66,0.31,0.22,0.22,0.94,0.41,0.4,0.19,-0.063,0.069,0.0083,0.0051,0.06,0.096,0.094,0.085,0.079,0.071,0.095,0.034,0.05,0.04,0.039,0.032,0.028,0.22,-0.39,-0.12,0.12,nan
closed_accounts,-0.35,0.016,-0.18,0.9,0.23,0.94,0.59,1.0,0.051,0.23,0.25,0.37,0.063,0.14,0.33,0.12,0.34,0.47,0.12,0.21,0.21,0.51,0.33,0.22,0.22,-0.041,0.031,0.024,0.065,0.12,0.14,0.12,0.095,0.085,0.072,0.084,0.052,0.046,0.036,0.036,0.032,0.045,0.15,-0.45,-0.18,0.18,nan
max_dpd,-0.033,0.0017,-0.25,0.072,0.31,0.027,0.095,0.051,1.0,-0.0049,0.03,-0.058,0.011,-0.056,0.068,-0.012,0.04,0.079,-0.14,0.26,0.26,0.084,0.045,0.0041,0.17,0.011,-0.018,0.0061,0.005,0.0095,0.012,0.01,0.0091,0.0064,0.0058,0.01,0.016,0.013,0.00089,0.0013,-0.00074,0.005,-0.00027,-0.034,-0.28,0.28,nan
credit_utilization,-0.19,0.02,-0.055,0.3,0.045,0.31,0.42,0.23,-0.0049,1.0,0.16,0.34,-0.063,0.21,0.12,0.22,0.29,0.21,0.15,0.091,0.091,0.44,0.19,0.91,0.12,-0.072,0.085,0.00099,-0.03,-0.0052,0.025,0.034,0.038,0.039,0.038,0.033,0.0064,0.023,0.024,0.025,0.023,0.0077,0.2,-0.2,-0.021,0.021,nan


In [32]:
ch_data.describe(percentiles=[0.01,0.05,0.10,0.25,0.5,0.75,0.9,0.95,0.99],datetime_is_numeric=True).transpose()

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
p_credit_score,3711888.00,675.42,82.21,0.00,500.00,525.00,550.00,625.00,700.00,725.00,775.00,775.00,775.00,775.00
total_accounts,3711888.00,14.63,13.37,0.00,1.00,1.00,2.00,4.00,10.00,21.00,38.00,47.00,47.00,47.00
negative_status_accounts,3711888.00,0.71,0.84,0.00,0.00,0.00,0.00,0.00,0.00,2.00,2.00,2.00,2.00,2.00
positive_status_accounts,3711888.00,10.95,8.05,0.00,0.00,1.00,2.00,4.00,9.00,19.00,23.00,23.00,23.00,23.00
open_accounts,3711888.00,5.07,4.00,0.00,0.00,0.00,1.00,2.00,4.00,7.00,12.00,14.00,14.00,14.00
closed_accounts,3711888.00,7.31,6.22,0.00,0.00,0.00,0.00,2.00,5.00,13.00,17.00,17.00,17.00,17.00
max_dpd,3711888.00,6.70,25.83,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,90.00,120.00,120.00
credit_utilization,3711888.00,16.15,32.60,0.00,0.00,0.00,0.00,0.00,0.00,2.22,86.15,100.00,100.00,100.00
enquiries_made,3711888.00,1.74,2.52,0.00,0.00,0.00,0.00,0.00,1.00,2.00,5.00,7.00,13.00,13.00
seasoning,3711888.00,2.34,2.46,0.00,0.00,0.00,0.00,0.00,2.00,4.00,7.00,8.00,8.00,8.00


In [33]:
'''Model Object'''
model_EMI = joblib.load('CH_EMI_RF_v1.2.pkl')
model_TXN = joblib.load('CH_TXN_RF_v1.1.pkl')
model_BL = joblib.load('CH_BL_RF_v1.1.pkl')
model_HL = joblib.load('CH_HL_RF_v1.1.pkl')
model_AL = joblib.load('CH_AL_RF_v1.1.pkl')

In [34]:
data_EMI=processed_dataset[list(model_EMI.feature_names_in_)]
data_TXN=processed_dataset[list(model_TXN.feature_names_in_)]
data_BL=processed_dataset[list(model_BL.feature_names_in_)]
data_HL=processed_dataset[list(model_HL.feature_names_in_)]
data_AL=processed_dataset[list(model_AL.feature_names_in_)]

In [35]:
test_fit = model_EMI.predict_proba(data_EMI)
y_pred=pd.DataFrame(test_fit)

In [36]:
y_pred_deciles = pd.concat([y_pred[1].reset_index(drop = True), processed_dataset[['user_id']].reset_index(drop = True)], axis = 1)
y_pred_deciles.rename(columns = {1:'predicted_value_EMI'}, inplace = True)

In [37]:
test_fit = model_TXN.predict_proba(data_TXN)
y_pred=pd.DataFrame(test_fit)

In [38]:
y_pred_deciles = pd.concat([y_pred[1].reset_index(drop = True),y_pred_deciles.reset_index(drop = True)], axis = 1)
y_pred_deciles.rename(columns = {1:'predicted_value_TXN'}, inplace = True)

In [39]:
test_fit = model_BL.predict_proba(data_BL)
y_pred=pd.DataFrame(test_fit)

In [40]:
y_pred_deciles = pd.concat([y_pred[1].reset_index(drop = True),y_pred_deciles.reset_index(drop = True)], axis = 1)
y_pred_deciles.rename(columns = {1:'predicted_value_BL'}, inplace = True)

In [41]:
test_fit = model_HL.predict_proba(data_HL)
y_pred=pd.DataFrame(test_fit)

In [42]:
y_pred_deciles = pd.concat([y_pred[1].reset_index(drop = True),y_pred_deciles.reset_index(drop = True)], axis = 1)
y_pred_deciles.rename(columns = {1:'predicted_value_HL'}, inplace = True)

In [43]:
test_fit = model_AL.predict_proba(data_AL)
y_pred=pd.DataFrame(test_fit)

In [44]:
y_pred_deciles = pd.concat([y_pred[1].reset_index(drop = True),y_pred_deciles.reset_index(drop = True)], axis = 1)
y_pred_deciles.rename(columns = {1:'predicted_value_AL'}, inplace = True)

In [45]:
y_pred_deciles['decile_EMI'] = pd.qcut(y_pred_deciles['predicted_value_EMI'].rank(method='first'), 10, labels=False)
y_pred_deciles['decile_EMI'] = y_pred_deciles['decile_EMI'].apply(lambda x: int(10 - x))
y_pred_deciles['decile_EMI'].value_counts()

5     371189
9     371189
10    371189
8     371189
1     371189
6     371189
2     371189
3     371189
7     371188
4     371188
Name: decile_EMI, dtype: int64

In [46]:
y_pred_deciles['decile_TXN'] = pd.qcut(y_pred_deciles['predicted_value_TXN'].rank(method='first'), 10, labels=False)
y_pred_deciles['decile_TXN'] = y_pred_deciles['decile_TXN'].apply(lambda x: int(10 - x))
y_pred_deciles['decile_TXN'].value_counts()

2     371189
9     371189
10    371189
3     371189
1     371189
8     371189
6     371189
5     371189
7     371188
4     371188
Name: decile_TXN, dtype: int64

In [47]:
y_pred_deciles['decile_BL'] = pd.qcut(y_pred_deciles['predicted_value_BL'].rank(method='first'), 10, labels=False)
y_pred_deciles['decile_BL'] = y_pred_deciles['decile_BL'].apply(lambda x: int(10 - x))
y_pred_deciles['decile_BL'].value_counts()

1     371189
9     371189
2     371189
6     371189
3     371189
8     371189
5     371189
10    371189
7     371188
4     371188
Name: decile_BL, dtype: int64

In [48]:
y_pred_deciles['decile_HL'] = pd.qcut(y_pred_deciles['predicted_value_HL'].rank(method='first'), 10, labels=False)
y_pred_deciles['decile_HL'] = y_pred_deciles['decile_HL'].apply(lambda x: int(10 - x))
y_pred_deciles['decile_HL'].value_counts()

2     371189
10    371189
1     371189
6     371189
3     371189
5     371189
8     371189
9     371189
7     371188
4     371188
Name: decile_HL, dtype: int64

In [49]:
y_pred_deciles['decile_AL'] = pd.qcut(y_pred_deciles['predicted_value_AL'].rank(method='first'), 10, labels=False)
y_pred_deciles['decile_AL'] = y_pred_deciles['decile_AL'].apply(lambda x: int(10 - x))
y_pred_deciles['decile_AL'].value_counts()

1     371189
6     371189
10    371189
5     371189
2     371189
8     371189
3     371189
9     371189
7     371188
4     371188
Name: decile_AL, dtype: int64

In [52]:
y_pred_deciles.head()

,predicted_value_AL,predicted_value_HL,predicted_value_BL,predicted_value_TXN,predicted_value_EMI,user_id,decile_EMI,decile_TXN,decile_BL,decile_HL,decile_AL
0,0.76,0.54,0.73,0.82,0.13,43,5,2,1,2,1
1,0.38,0.23,0.33,0.31,0.03,60,9,9,7,7,7
2,0.42,0.36,0.34,0.42,0.02,71,10,7,7,4,6
3,0.16,0.11,0.21,0.21,0.06,93,8,10,9,10,10
4,0.77,0.73,0.67,0.69,0.07,130,7,3,2,1,1


In [50]:
y_pred_deciles.to_csv('CH_Procucts_deciled.csv')

In [51]:
from pyfiglet import Figlet
import datetime
custom_fig = Figlet(font='starwars', width=130)
print(custom_fig.renderText('>>>     Prediction  Completed      <<<'))
custom_fig = Figlet(font='digital', width=120, justify='center')
print(custom_fig.renderText(str(datetime.datetime.now())))

___  ___  ___                  .______   .______       _______  _______   __    ______ .___________. __    ______   .__   __.    
\  \ \  \ \  \                 |   _  \  |   _  \     |   ____||       \ |  |  /      ||           ||  |  /  __  \  |  \ |  |    
 \  \ \  \ \  \                |  |_)  | |  |_)  |    |  |__   |  .--.  ||  | |  ,----'`---|  |----`|  | |  |  |  | |   \|  |    
  >  > >  > >  >               |   ___/  |      /     |   __|  |  |  |  ||  | |  |         |  |     |  | |  |  |  | |  . `  |    
 /  / /  / /  /                |  |      |  |\  \----.|  |____ |  '--'  ||  | |  `----.    |  |     |  | |  `--'  | |  |\   |    
/__/ /__/ /__/                 | _|      | _| `._____||_______||_______/ |__|  \______|    |__|     |__|  \______/  |__| \__|    
                                                                                                                                 
  ______   ______   .___  ___. .______    __       _______ .___________. _______  _______ 